In [15]:
IDENTIFIER = "beethoven-op67"
NUM = 4

import matplotlib.pyplot as plt
import json
import networkx as nx
import pandas as pd
import numpy as np
from dwave.system import DWaveSampler, FixedEmbeddingComposite
from dwave.samplers import SteepestDescentSolver, SimulatedAnnealingSampler
import dwave.inspector

from plots import plotHistogram
from qubo import createBQM
from samples import totalEntropy, overlaps, duplicates

picklePath = f"../Pickles/{IDENTIFIER}/{IDENTIFIER}_"
numPath = f"../Pickles/{IDENTIFIER}/{NUM}/{IDENTIFIER}_{NUM}_"
dataPath = f"../Data/{IDENTIFIER}/{NUM}/{IDENTIFIER}_{NUM}_"

phrases = pd.read_csv(picklePath + "phrases.csv", index_col=[0,1])
G = nx.read_graphml(picklePath + "graph.graphml")
instruments = json.load(open(numPath + "instruments.json"))
embedding = json.load(open(numPath + "embedding.json", "r"))

In [16]:
cs = 30
at = 200

def recordEnergyReads(readRange, bqm, sampler, filepath: str, header: bool = False, quantum: bool = False):
    
    file = pd.DataFrame()
    for r in readRange:

        if quantum:
            sampleset = sampler.sample(bqm, num_reads=r, chain_strength=cs, annealing_time=at,label=f"{IDENTIFIER}_{r}_{cs}_{at}")
        else:
            sampleset = sampler.sample(bqm, num_reads=r)
        
        sample = sampleset.first

        if quantum:
            new_row = pd.DataFrame({
                "Total reads": [r],
                "Time": [sampleset.info["timing"]["qpu_sampling_time"]],
                "Chain strength": [cs],
                "Chain break fraction": [sample.chain_break_fraction],
                "Lowest energy": [bqm.energy(sample.sample)],
                "Total entropy": [totalEntropy(sample.sample, G)],
                "Overlaps": [overlaps(sample.sample, G)],
                "Duplicates": [duplicates(sample.sample, G)]
                })
        else:
            new_row = pd.DataFrame({
                "Total reads": [r],
                "Lowest energy": [bqm.energy(sample.sample)],
                "Total entropy": [totalEntropy(sample.sample, G)],
                "Time": [sampleset.info["timing"]["sampling_ns"]/1000],
                "Overlaps": [overlaps(sample.sample, G)],
                "Duplicates": [duplicates(sample.sample, G)]
                })
    
        print(f"{r} reads recorded!")
        file = pd.concat([file, new_row])

    file.to_csv(filepath, index=False, mode="a", header=header)

bqm = createBQM(G, phrases, instruments, 6, 12, 1) # CHANGE
qa = FixedEmbeddingComposite(DWaveSampler(), embedding)
sa = SimulatedAnnealingSampler()
sd = SteepestDescentSolver()

# 200 to 2000 repeated
readRange = np.arange(2000,2200,200)

for _ in range(2):
    recordEnergyReads(readRange, bqm, qa, dataPath+"qa-reads.csv", quantum=True, header=False)
    #recordEnergyReads(readRange, bqm, sa, dataPath+"sa-reads.csv", quantum=False, header=False)
    #recordEnergyReads(readRange, bqm, sd, dataPath+"sd-reads.csv", quantum=False, header=False)

2000 reads recorded!
2000 reads recorded!


In [ ]:
reads = 1000

qa = qa.sample(bqm, num_reads=reads, label=f"Haydn_{reads}").to_pandas_dataframe(True)
sd = SteepestDescentSolver().sample(bqm, num_reads=reads).to_pandas_dataframe(True)
sa = SimulatedAnnealingSampler().sample(bqm, num_reads=reads).to_pandas_dataframe(True)

#random = RandomSampler().sample(bqm, num_reads=1000).to_pandas_dataframe(True)

plt.subplot(3, 1, 1)
plt.title("Annealing")
plotHistogram(qa)
plt.subplot(3, 1, 2)
plt.title("Steepest descent")
plotHistogram(sd)
plt.subplot(3, 1, 3)
plt.title("Simulated annealing")
plotHistogram(sa)

plt.show()

In [ ]:
sampleset = qa.sample(bqm, num_reads=1)
print(sampleset.info["timing"])
#plotSampleGraph(sampleset.first.sample, G, instruments)
#plotArrangement(sampleset.first.sample, phrases, instruments)